In [1]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM
from keras.layers import  Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import datetime
from datetime import time
import numpy as np

2025-05-26 08:03:04.366238: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-26 08:03:04.369777: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-26 08:03:04.380255: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748246584.397786     400 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748246584.403817     400 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748246584.417075     400 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [11]:
def calculate_mape(actual, predicted):
    """
    计算 MAPE（平均绝对百分比误差）

    参数:
    
    
    actual (array-like): 实际值数组。
    predicted (array-like): 预测值数组。

    返回:
    float: MAPE 值。
    """
    # 将输入转换为 numpy 数组
    actual = np.array(actual)
    predicted = np.array(predicted)

    # 避免除以零的情况
    if np.any(actual == 0):
        raise ValueError("实际值中包含零，无法计算 MAPE。")
    # 计算 MAPE
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return mape

In [12]:
# set up the interval need to forecast here
unique_times = [ datetime.time(10, 0),
datetime.time(10, 30), datetime.time(11, 0), datetime.time(11, 30),
datetime.time(12, 0), datetime.time(12, 30), datetime.time(13, 0),
datetime.time(13, 30), datetime.time(14, 0), datetime.time(14, 30),
datetime.time(15, 0), datetime.time(15, 30), datetime.time(16, 0),
datetime.time(16, 30), datetime.time(17, 0), datetime.time(17, 30),datetime.time(18,0),datetime.time(18,30),datetime.time(19,0),datetime.time(19,30),datetime.time(20,0),datetime.time(20,30)
]

In [13]:

#data = pd.read_excel('workspace/MidWest_WFM_Stat_2025_05_16.xlsx')
data = pd.read_excel('MidWest_WFM_Stat_2025_05_16.xlsx')
pd.set_option('display.max_rows', 100)
data.columns = ['conversation_start_interval_tmst', 'Time', 'offered', 'actans',
       'actabn', 'absActHt', 'absActSa', 'parent', 'child', 'fiscalDate',
       'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']

data = data[~data .offered.isna()]

data = data[data .offered != 0]
# Month one and half need to set up 
data_=data
data = data[data.conversation_start_interval_tmst<=pd.to_datetime('{} 00:00:00'.format('2025-01-15'))]
#holiday_data = pd.read_excel(r'workspace/holiday.xlsx')
holiday_data = pd.read_excel(r'holiday.xlsx')
holiday_data['~is_holiday'] = 0
Holiday_name = ['Christmas Day', 'Columbus Day',
       'Independence Day', 'Labor Day', 'Martin Luther King Jr. Day',
       'Memorial Day', "New Year's Day", "Presidents' Day", 'Thanksgiving Day',
       'Veterans Day']
holiday_data['date'] = holiday_data.Date.dt.date
data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
data['date'] = data.datetime.dt.date

data.datetime = data.datetime.dt.floor('T')
data['datetime'] = data['datetime'].apply(
    lambda x: x.ceil('30T') if x.minute == 29 else x
)
data['datetime'] = data['datetime'].apply(
    lambda x: x.ceil('H') if x.minute == 59 else x
)
data['time'] = data.datetime.dt.time
data = data.sort_values('datetime').reset_index(drop = True)
data = data.sort_values(by='datetime')

# 获取所有唯一的时间点（如每天的09:00, 09:30等）
#unique_times = data['time'].unique()

# 获取所有唯一的日期
unique_dates = data['date'].unique()
data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)
data = data[data['~is_holiday'] == 1]

holiday_datetime = holiday_data.date.to_numpy()

start_time = 35+23 # 58 how many days in advance
end_time = start_time+36 #36 is the train length


add_column = ['actans','actabn','absActHt','absActSa']

add_columns =  [f'actans_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'actabn_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'absActHt_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'absActSa_{i}' for i in range(start_time+1,end_time+1) ]

matrix = []


/tmp/ipykernel_3999269/72552902.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
/tmp/ipykernel_3999269/72552902.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data.datetime.dt.date
/tmp/ipykernel_3999269/72552902.py:25: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data.datetime = data.datetime.dt.floor('T')
/tmp/ipykernel_3999269/72552902.py:25: SettingW

In [9]:
len(add_columns)

144

In [40]:
matrix = []

# 遍历每个时间点
for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
    # 过滤出当前时间点的数据
    time_data = data[data['time'] == time].set_index('date')['offered']    
    full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max(), freq='B')  # 仅工作日
    full_dates = full_dates.difference(holiday_datetime)
    time_data = time_data.groupby(time_data.index).sum()
    time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
    
    
    add_data = data[data['time'] == time].set_index('date')[add_column]
    add_data = add_data.groupby(add_data.index).sum()
    add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
    
    
    time_data1 = data[data['time'] == time1].set_index('date')['offered']    
    time_data1 = time_data1.groupby(time_data1.index).sum()
    time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
    
    time_data2 = data[data['time'] == time2].set_index('date')['offered']    
    time_data2 = time_data2.groupby(time_data2.index).sum()
    time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
    df=time_data.to_frame(name='offer')
    df_last_year = df.copy()
    df_last_year.index = df_last_year.index + pd.DateOffset(years=1)
    df_last_year = df_last_year.rename(columns={'offer': 'offered_last_year'}) 
    # Step 3: Join on index
    result = df.join(df_last_year, how='left')
    year_data=result['offered_last_year']  
    #year_data = time_data.shift(251)
    break
        

In [18]:
year_data=time_data.copy()
year_data.index = year_data.index + pd.DateOffset(years=1)
year_data.columns=['last+year']

In [19]:
year_data

2022-12-21    114.0
2022-12-22    110.0
2022-12-23    102.0
2022-12-24     33.0
2022-12-27    102.0
              ...  
2026-01-08    128.0
2026-01-09    127.0
2026-01-10    137.0
2026-01-13    154.0
2026-01-14    161.0
Name: offered, Length: 772, dtype: float64

In [20]:
# Assume time_data has datetime index and 'offered' column
# Step 1: Shift the index by 1 year
time_data_last_year = time_data.copy()
time_data_last_year.index = time_data_last_year.index + pd.DateOffset(years=1)
# Step 2: Join the original with the shifted
result = time_data.join(time_data_last_year, how='left')

AttributeError: 'Series' object has no attribute 'join'

In [25]:
type(time_data)

pandas.core.series.Series

In [26]:
time_data_=time_data.to_frame(name='offer')

In [33]:
df=time_data.to_frame(name='offer')
df_last_year = df.copy()
df_last_year.index = df_last_year.index + pd.DateOffset(years=1)
df_last_year = df_last_year.rename(columns={'offer': 'offered_last_year'})

# Step 3: Join on index
result = df.join(df_last_year, how='left')
year_data=result['offered_last_year']

In [36]:
result['offered_last_year'].isna().sum()

np.int64(420)

In [37]:
len(result['offered_last_year'])

772

In [38]:
year_data=result['offered_last_year']

In [39]:
type(year_data)

pandas.core.series.Series

In [41]:
year_data

2021-12-21      NaN
2021-12-22      NaN
2021-12-23      NaN
2021-12-24      NaN
2021-12-27      NaN
              ...  
2025-01-08    120.0
2025-01-09    160.0
2025-01-10    114.0
2025-01-13      NaN
2025-01-14      NaN
Name: offered_last_year, Length: 772, dtype: float64